Прежде чем сделать модель, которая будет подсказывать, каких героев лучше выбрать, решим схожую задачу. По заданному пику будем предсказывать вероятность победы той или иной команды. Для начала сделаем линейную модель, для этого отлично подходит логистическая регрессия. Но прежде всего подготовим наш датасет.


In [1]:
import json
from matplotlib.colors import ListedColormap
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd


In [2]:
with open('matches.json', 'rb') as f:
   json_data = f.readlines()
dict_data=json.loads(json_data[0])
data = pd.DataFrame.from_dict(dict_data).T
data = data[data['game_mode'] == 'All Pick']
data.head()

,game_mode,win,duration,radiant,dire,skill
5570729617,All Pick,Dire,"{'m': 32, 's': 49}","[Riki, Queen of Pain, Rubick, Void Spirit, Pudge]","[Zeus, Tinker, Legion Commander, Treant Protec...",normal_skill
5570726601,All Pick,Dire,"{'m': 34, 's': 21}","[Anti-Mage, Tinker, Zeus, Pudge, Skywrath Mage]","[Lone Druid, Pugna, Puck, Venomancer, Riki]",normal_skill
5570726975,All Pick,Radiant,"{'m': 33, 's': 58}","[Abaddon, Crystal Maiden, Nature's Prophet, Te...","[Spectre, Kunkka, Tiny, Queen of Pain, Clockwerk]",normal_skill
5570729540,All Pick,Dire,"{'m': 32, 's': 22}","[Void Spirit, Zeus, Weaver, Timbersaw, Nature'...","[Ember Spirit, Slardar, Pudge, Juggernaut, Jak...",normal_skill
5570729780,All Pick,Dire,"{'m': 32, 's': 8}","[Sand King, Templar Assassin, Beastmaster, Emb...","[Doom, Witch Doctor, Crystal Maiden, Phantom A...",normal_skill


In [3]:
with open('heroes.json', 'rb') as f:
   json_heroes = f.readlines()
dict_heroes=json.loads(json_heroes[0])
heroes = pd.DataFrame.from_dict(dict_heroes)
heroes.head()

,Abaddon,Alchemist,Ancient Apparition,Anti-Mage,Arc Warden,Axe,Bane,Batrider,Beastmaster,Bloodseeker,...,Viper,Visage,Void Spirit,Warlock,Weaver,Windranger,Winter Wyvern,Witch Doctor,Wraith King,Zeus
role,"[Melee, Carry, Durable, Support]","[Melee, Carry, Disabler, Durable, Initiator, N...","[Ranged, Disabler, Nuker, Support]","[Melee, Carry, Escape, Nuker]","[Ranged, Carry, Escape, Nuker]","[Melee, Disabler, Durable, Initiator, Jungler]","[Ranged, Disabler, Durable, Nuker, Support]","[Ranged, Disabler, Escape, Initiator, Jungler]","[Melee, Disabler, Durable, Initiator, Nuker]","[Melee, Carry, Disabler, Initiator, Jungler, N...",...,"[Ranged, Carry, Disabler, Durable, Initiator]","[Ranged, Disabler, Durable, Nuker, Pusher, Sup...","[Melee, Carry, Disabler, Escape, Nuker]","[Ranged, Disabler, Initiator, Support]","[Ranged, Carry, Escape]","[Ranged, Carry, Disabler, Escape, Nuker, Support]","[Ranged, Disabler, Nuker, Support]","[Ranged, Disabler, Nuker, Support]","[Melee, Carry, Disabler, Durable, Initiator, S...","[Ranged, Nuker]"
main_attribute,strength,strength,intelligence,agility,agility,strength,intelligence,intelligence,strength,agility,...,agility,intelligence,intelligence,intelligence,agility,intelligence,intelligence,intelligence,strength,intelligence
stats,"{'str': 23, 'agi': 23, 'int': 18}","{'str': 25, 'agi': 22, 'int': 25}","{'str': 20, 'agi': 20, 'int': 23}","{'str': 23, 'agi': 24, 'int': 12}","{'str': 25, 'agi': 16, 'int': 24}","{'str': 25, 'agi': 20, 'int': 18}","{'str': 22, 'agi': 22, 'int': 22}","{'str': 28, 'agi': 15, 'int': 22}","{'str': 23, 'agi': 18, 'int': 16}","{'str': 24, 'agi': 22, 'int': 21}",...,"{'str': 21, 'agi': 21, 'int': 17}","{'str': 22, 'agi': 11, 'int': 22}","{'str': 22, 'agi': 19, 'int': 24}","{'str': 26, 'agi': 10, 'int': 25}","{'str': 16, 'agi': 14, 'int': 16}","{'str': 18, 'agi': 17, 'int': 18}","{'str': 24, 'agi': 16, 'int': 26}","{'str': 18, 'agi': 13, 'int': 22}","{'str': 22, 'agi': 18, 'int': 18}","{'str': 21, 'agi': 11, 'int': 22}"
inc_stats,"{'str': 3.0, 'agi': 1.5, 'int': 2.0}","{'str': 2.7, 'agi': 1.5, 'int': 1.8}","{'str': 1.9, 'agi': 2.2, 'int': 3.4}","{'str': 1.3, 'agi': 2.8, 'int': 1.8}","{'str': 3.0, 'agi': 2.5, 'int': 2.6}","{'str': 3.4, 'agi': 2.2, 'int': 1.6}","{'str': 2.6, 'agi': 2.6, 'int': 2.6}","{'str': 2.9, 'agi': 1.8, 'int': 2.9}","{'str': 2.9, 'agi': 1.6, 'int': 1.9}","{'str': 2.7, 'agi': 3.4, 'int': 2.0}",...,"{'str': 2.4, 'agi': 2.5, 'int': 2.0}","{'str': 2.8, 'agi': 1.3, 'int': 2.9}","{'str': 2.6, 'agi': 2.2, 'int': 3.1}","{'str': 3.0, 'agi': 1.0, 'int': 3.1}","{'str': 2.0, 'agi': 3.1, 'int': 2.0}","{'str': 3.0, 'agi': 1.4, 'int': 3.6}","{'str': 2.6, 'agi': 1.9, 'int': 3.6}","{'str': 2.3, 'agi': 1.4, 'int': 3.3}","{'str': 3.0, 'agi': 1.7, 'int': 1.6}","{'str': 2.1, 'agi': 1.2, 'int': 3.3}"
move_speed,325,305,285,310,285,310,305,290,305,300,...,275,285,295,290,275,290,285,300,315,310


In [155]:
parametrs = []
for hero in heroes.columns:
    for role in heroes[hero]['role']:
        if role not in parametrs:
            parametrs.append(role)
for parametr in ['str', 'agi', 'int', 'start_str', 'start_agi', 'start_int']:
    parametrs.append(parametr)
for parametr in ['move_speed', 'armor', 'attack_speed', 'attack_point', 'damage', 'winrate', 'kda']:
    parametrs.append(parametr)
for parametr in heroes['Abaddon']['cnt_skills']:
    parametrs.append(parametr)
tmp = parametrs.copy()
for param in tmp:
    parametrs.append('enemy_' + param)
parametrs.append('skill')
parametrs.append('win')

In [156]:
indexes = list(data.index)

In [157]:
m_data = pd.DataFrame(np.zeros((len(indexes), len(parametrs))), 
                      index=indexes,
                      columns=parametrs)

In [158]:
import time

In [159]:
#3-radiant, 4-dire
start_time = time.time()
for ind in data.index:
    m_data.loc[ind]['win'] = 1 if data.loc[ind]['win'] == 'Radiant' else 0
    
    if data.loc[ind]['skill'] == 'normal_skill':
        m_data.loc[ind]['skill'] = 1
    elif data.loc[ind]['skill'] == 'high_skill':
        m_data.loc[ind]['skill'] = 2
    else:
        m_data.loc[ind]['skill'] = 3
    
    for hero in data.loc[ind]['radiant']:
        for role in heroes[hero]['role']:
            m_data.loc[ind][role] += 1
        
        for stat in heroes[hero]['stats']:
            m_data.loc[ind]['start_' + stat] += heroes[hero]['stats'][stat]
        
        for stat in heroes[hero]['inc_stats']:
            m_data.loc[ind][stat] += heroes[hero]['inc_stats'][stat]
            
        m_data.loc[ind]['move_speed'] += heroes[hero]['move_speed']
        
        for stat in ['armor', 'attack_speed', 'attack_point', 'kda']:
            m_data.loc[ind][stat] += heroes[hero][stat]
            
        if data.loc[ind]['skill'] == 'normal_skill':
            m_data.loc[ind]['winrate'] += heroes[hero]['winrate']['2K-3K MMR']
        elif data.loc[ind]['skill'] == 'high_skill':
            m_data.loc[ind]['winrate'] += heroes[hero]['winrate']['4K-5K MMR']
        else:
            m_data.loc[ind]['winrate'] += heroes[hero]['winrate']['>5K MMR']
            
        m_data.loc[ind]['damage'] += (heroes[hero]['damage']['min'] + heroes[hero]['damage']['max']) / 2
        
        for stat in heroes[hero]['cnt_skills']:
            m_data.loc[ind][stat] += heroes[hero]['cnt_skills'][stat]
            
    for hero in data.loc[ind]['dire']:
        for role in heroes[hero]['role']:
            m_data.loc[ind]['enemy_' + role] += 1
        
        for stat in heroes[hero]['stats']:
            m_data.loc[ind]['enemy_' + 'start_' + stat] += heroes[hero]['stats'][stat]
        
        for stat in heroes[hero]['inc_stats']:
            m_data.loc[ind]['enemy_' + stat] += heroes[hero]['inc_stats'][stat]
            
        m_data.loc[ind]['enemy_' + 'move_speed'] += heroes[hero]['move_speed']
        
        for stat in ['armor', 'attack_speed', 'attack_point', 'kda']:
            m_data.loc[ind]['enemy_' + stat] += heroes[hero][stat]
            
        if data.loc[ind]['skill'] == 'normal_skill':
            m_data.loc[ind]['enemy_' + 'winrate'] += heroes[hero]['winrate']['2K-3K MMR']
        elif data.loc[ind]['skill'] == 'high_skill':
            m_data.loc[ind]['enemy_' + 'winrate'] += heroes[hero]['winrate']['4K-5K MMR']
        else:
            m_data.loc[ind]['enemy_' + 'winrate'] += heroes[hero]['winrate']['>5K MMR']
            
        m_data.loc[ind]['enemy_' + 'damage'] += (heroes[hero]['damage']['min'] + heroes[hero]['damage']['max']) / 2
        
        for stat in heroes[hero]['cnt_skills']:
            m_data.loc[ind]['enemy_' + stat] += heroes[hero]['cnt_skills'][stat]
            
        
            
        
        
    
print("--- %s seconds ---" % (time.time() - start_time))       
m_data.T

--- 9907.039332389832 seconds ---


,5570729617,5570726601,5570726975,5570729540,5570729780,5570725038,5570724931,5570730958,5570707585,5570727531,...,5579876845,5579886344,5579879245,5579869391,5579850801,5579886232,5579919719,5579821089,5579882745,5579845783
Melee,3.0,2.0,2.0,2.0,3.0,2.0,0.0,3.0,3.0,3.0,...,3.0,3.0,2.0,2.0,0.0,2.0,1.0,1.0,4.0,3.0
Carry,3.0,2.0,4.0,3.0,2.0,3.0,3.0,2.0,4.0,3.0,...,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0
Durable,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,3.0,3.0,...,3.0,2.0,2.0,2.0,1.0,3.0,1.0,0.0,4.0,3.0
Support,1.0,1.0,2.0,0.0,2.0,1.0,2.0,1.0,2.0,1.0,...,2.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0,3.0
Disabler,4.0,2.0,2.0,1.0,4.0,3.0,2.0,2.0,5.0,3.0,...,5.0,3.0,4.0,4.0,3.0,2.0,2.0,2.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
enemy_attributes,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
enemy_silence,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,4.0,0.0
enemy_dispel,4.0,0.0,0.0,3.0,1.0,0.0,2.0,2.0,0.0,1.0,...,1.0,1.0,1.0,4.0,0.0,1.0,2.0,1.0,3.0,2.0
skill,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


Делаем вторую выборку
==

**Будем ориентироваться только на VeryHighSkill, ибо зачастую у этих игроков более продуман пик, что позволяет легче проявить зависимости между признаками и результатом**

In [4]:
data_vhs = data[data['skill'] == 'very_high_skill']
data_vhs.head()

,game_mode,win,duration,radiant,dire,skill
5570720824,All Pick,Radiant,"{'m': 39, 's': 0}","[Nature's Prophet, Necrophos, Pangolier, Anti-...","[Bounty Hunter, Magnus, Batrider, Shadow Shama...",very_high_skill
5570737832,All Pick,Radiant,"{'m': 28, 's': 24}","[Clinkz, Zeus, Spectre, Pudge, Mirana]","[Terrorblade, Snapfire, Disruptor, Lina, Mars]",very_high_skill
5570740694,All Pick,Dire,"{'m': 26, 's': 39}","[Dazzle, Wraith King, Doom, Mirana, Mars]","[Night Stalker, Lone Druid, Enigma, Winter Wyv...",very_high_skill
5570717826,All Pick,Radiant,"{'m': 40, 's': 44}","[Faceless Void, Queen of Pain, Dark Seer, Mira...","[Tidehunter, Ember Spirit, Terrorblade, Ogre M...",very_high_skill
5570711957,All Pick,Radiant,"{'m': 44, 's': 12}","[Invoker, Earthshaker, Keeper of the Light, Wi...","[Grimstroke, Pudge, Anti-Mage, Zeus, Void Spirit]",very_high_skill


In [5]:
indexes = list(data_vhs.index)
columns = list(heroes.columns)
for hero in columns.copy():
    columns.append('enemy_' + hero)
columns.append('win')

In [6]:
df_vhs = pd.DataFrame(np.zeros((len(indexes), len(columns))), 
                      index=indexes,
                      columns=columns)

In [7]:
%%time
for ind in indexes:
    df_vhs.loc[ind]['win'] = 1 if data_vhs.loc[ind]['win'] == 'Radiant' else 0
    
    for hero in data_vhs.loc[ind]['radiant']:
        df_vhs.loc[ind][hero] = 1
    for hero in data_vhs.loc[ind]['dire']:
        df_vhs.loc[ind]['enemy_' + hero] = 1

Wall time: 48.5 s


In [8]:
df_vhs

,Abaddon,Alchemist,Ancient Apparition,Anti-Mage,Arc Warden,Axe,Bane,Batrider,Beastmaster,Bloodseeker,...,enemy_Visage,enemy_Void Spirit,enemy_Warlock,enemy_Weaver,enemy_Windranger,enemy_Winter Wyvern,enemy_Witch Doctor,enemy_Wraith King,enemy_Zeus,win
5570720824,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5570737832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5570740694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5570717826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5570711957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579886232,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5579919719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5579821089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5579882745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Попробуем выборку только из героев
==

In [9]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.preprocessing import StandardScaler, normalize
import warnings
warnings.filterwarnings('ignore')

In [32]:
def gridL1(train, label):
    classifier = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
    parameters_grid = {
        'penalty' : ['l1'],
        'solver' : ['liblinear', 'saga'],
        'C' : np.linspace(0.1, 10, num = 10),
    }
    cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
    grid_cv.fit(train, label)
    print(grid_cv.best_score_)
    print(grid_cv.best_params_)

In [33]:
def gridL2(train, label):
    classifier = LogisticRegression(penalty='l2', random_state=42)
    parameters_grid = {
    'penalty' : ['l2'],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'C' : np.linspace(0.1, 10, num = 10),
    }
    cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
    grid_cv.fit(train, label)
    print(grid_cv.best_score_)
    print(grid_cv.best_params_)

In [34]:
def gridC(penalty, train, label, solver, C):
    classifier = LogisticRegression(penalty=penalty, solver=solver, random_state=42)
    parameters_grid = {
    'C' : C
    }
    cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state=42)
    grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
    grid_cv.fit(train, label)
    print(grid_cv.best_score_)
    print(grid_cv.best_params_)

In [35]:
X = df_vhs.values[:,0:-1]
y = df_vhs.values[:,-1]
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, test_size = 0.2,random_state = 42)
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state=42)

Все признаки бинарные, поэтому масштабировать не имеет смысла

**L1**

In [36]:
%%time
gridL1(train_data, train_labels)

0.5627833972793861
{'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Wall time: 6min 45s


In [37]:
%%time
gridC('l1', train_data, train_labels, 'saga', np.linspace(0.0001, 0.2, num = 20))

0.5634461109173352
{'C': 0.13687368421052631}
Wall time: 7min 45s


In [38]:
classifier = LogisticRegressionCV(penalty='l1', solver='liblinear',Cs=[0.1369], cv=cv)
classifier.fit(train_data, train_labels)
print(classifier.score(test_data, test_labels))
print(classifier.scores_[1].mean())
print('proba', proba_metric(classifier, test_data, test_labels))

0.5638950892857143
0.563184513428671
proba 0.4865348163430472


**L2**

In [39]:
%%time
gridL2(train_data, train_labels)

0.5617021276595745
{'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Wall time: 12min 19s


In [40]:
%%time
gridC('l2', train_data, train_labels, 'lbfgs', np.linspace(0.0001, 0.2, num = 20))

0.5638297872340424
{'C': 0.021142105263157896}
Wall time: 41.1 s


In [41]:
classifier = LogisticRegressionCV(penalty='l2', solver='lbfgs',Cs=[0.0211], cv=cv)
classifier.fit(train_data, train_labels)
print(classifier.score(test_data, test_labels))
print(classifier.scores_[1].mean())
print('proba', proba_metric(classifier, test_data, test_labels))

0.5661272321428571
0.5638646668991978
proba 0.48617758139667117


Попробуем добавить средний winrate
==

In [42]:
df_vhs_wr = df_vhs.copy()

In [43]:
df_vhs_wr['winrate'] = 0.
df_vhs_wr['enemy_winrate'] = 0.

for ind in indexes:
    for hero in data_vhs.loc[ind]['radiant']:
        df_vhs_wr.loc[ind]['winrate'] += (heroes[hero]['winrate']['>5K MMR'] - 50) / 5
    for hero in data_vhs.loc[ind]['dire']:
        df_vhs_wr.loc[ind]['enemy_winrate'] += (heroes[hero]['winrate']['>5K MMR'] - 50) / 5
cols = df_vhs_wr.columns.tolist()
cols[-1], cols[-3] = cols[-3], cols[-1]
df_vhs_wr = df_vhs_wr[cols]

In [44]:
df_vhs_wr

,Abaddon,Alchemist,Ancient Apparition,Anti-Mage,Arc Warden,Axe,Bane,Batrider,Beastmaster,Bloodseeker,...,enemy_Warlock,enemy_Weaver,enemy_Windranger,enemy_Winter Wyvern,enemy_Witch Doctor,enemy_Wraith King,enemy_Zeus,enemy_winrate,winrate,win
5570720824,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.49980,-0.21836,1.0
5570737832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.56780,1.20670,1.0
5570740694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.21698,-0.37710,0.0
5570717826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.52436,1.11316,1.0
5570711957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.64984,-0.61638,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579886232,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.51630,0.39550,1.0
5579919719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.76604,1.00576,0.0
5579821089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.17342,0.76868,1.0
5579882745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.06572,0.62990,0.0


In [45]:
X = df_vhs_wr.values[:,0:-1]
y = df_vhs_wr.values[:,-1]
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, test_size = 0.2,random_state = 42)

**L1**

In [49]:
%%time
gridL1(train_data, train_labels)

0.5689047785141262
{'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Wall time: 11min 11s


In [50]:
%%time
gridC('l1', train_data, train_labels, 'saga', np.linspace(0.0001, 0.2, 20))

0.570160446459714
{'C': 0.05270526315789474}
Wall time: 6min 40s


In [51]:
classifier = LogisticRegressionCV(penalty='l1', solver='saga', Cs=[0.0527])
classifier.fit(train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(test_data, test_labels))
print('proba', proba_metric(classifier, test_data, test_labels))

0.5672881063383828
0.5560825892857143
proba 0.48693640846352615


**L2**

In [52]:
%%time
gridL2(train_data, train_labels)

0.5642832228810603
{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
Wall time: 10min 43s


In [53]:
%%time
gridC('l2', train_data, train_labels, 'newton-cg', np.linspace(0.0001, 0.2, 20))

0.5689396581792814
{'C': 0.010621052631578948}
Wall time: 4min 15s


In [55]:
classifier = LogisticRegressionCV(penalty='l2', solver='newton-cg', Cs=[0.0106])
classifier.fit(train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(test_data, test_labels))
print('proba', proba_metric(classifier, test_data, test_labels))

0.5671135619962169
0.5594308035714286
proba 0.4859241528765256


Попробуем добавить винрейт текущей команды против конкретных оппонентов
==

**Также добавис урон по строениям**

In [56]:
df_vhs_wr_team = df_vhs_wr.copy()
df_vhs_wr_team['team_wr'] = 0.
df_vhs_wr_team['dmg_tower_minute'] = 0.
df_vhs_wr_team['enemy_dmg_tower_minute'] = 0.

for ind in indexes:
    for hero in data_vhs.loc[ind]['radiant']:
        df_vhs_wr_team.loc[ind]['dmg_tower_minute'] += heroes[hero]['dmg_tower_minute'] / 5
        for enemy_hero in data_vhs.loc[ind]['dire']:
            df_vhs_wr_team.loc[ind]['team_wr'] += heroes[hero]['advantages'][enemy_hero] / 25
    for en_hero in data_vhs.loc[ind]['dire']:
        df_vhs_wr_team.loc[ind]['enemy_dmg_tower_minute'] += heroes[en_hero]['dmg_tower_minute'] / 5

cols = df_vhs_wr_team.columns.tolist()
cols[-1], cols[-4] = cols[-4], cols[-1]
df_vhs_wr_team = df_vhs_wr_team[cols]

In [57]:
df_vhs_wr_team

,Abaddon,Alchemist,Ancient Apparition,Anti-Mage,Arc Warden,Axe,Bane,Batrider,Beastmaster,Bloodseeker,...,enemy_Winter Wyvern,enemy_Witch Doctor,enemy_Wraith King,enemy_Zeus,enemy_winrate,winrate,enemy_dmg_tower_minute,team_wr,dmg_tower_minute,win
5570720824,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.49980,-0.21836,56.19738,-0.006448,88.87212,1.0
5570737832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.56780,1.20670,70.52992,0.500100,69.95428,1.0
5570740694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.21698,-0.37710,86.07426,0.026108,58.34932,0.0
5570717826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.52436,1.11316,68.88430,0.002732,55.93950,1.0
5570711957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-1.64984,-0.61638,50.22402,0.174228,55.39800,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579886232,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.51630,0.39550,73.94524,0.094208,85.44546,1.0
5579919719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.76604,1.00576,55.59968,0.359360,51.36418,0.0
5579821089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-1.17342,0.76868,49.59128,-0.440896,61.41216,1.0
5579882745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.06572,0.62990,102.48850,0.156784,42.99414,0.0


In [58]:
X = df_vhs_wr_team.values[:,0:-1]
y = df_vhs_wr_team.values[:,-1]
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, test_size = 0.2,random_state = 42)

**L1**

In [116]:
%%time
gridL1(train_data, train_labels)

0.5815486571328915
{'C': 1.2000000000000002, 'penalty': 'l1', 'solver': 'liblinear'}
Wall time: 23min 11s


In [117]:
%%time
gridC('l1', train_data, train_labels, 'liblinear', np.linspace(0.2, 1.3, 10))

0.5798918730380189
{'C': 0.2}
Wall time: 2min 3s


In [47]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2)
classifier = LogisticRegressionCV(penalty='l1', solver='liblinear', Cs=[0.2])
classifier.fit(train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(test_data, test_labels))
print('proba', proba_metric(classifier, test_data, test_labels))

0.582392303330068
0.5853794642857143
proba 0.47655987196337335


In [123]:
classifier.coef_[0][-4:]

array([ 1.91840834e-01, -3.61225698e-04, -1.10467693e+00,  2.27301107e-03])

**L2**

In [124]:
%%time
gridL2(train_data, train_labels)

0.5836588768747819
{'C': 6.7, 'penalty': 'l2', 'solver': 'saga'}
Wall time: 20min 52s


In [125]:
gridC('l2', train_data, train_labels, 'saga', np.linspace(6, 7, 10))

0.582106731775375
{'C': 6.555555555555555}


In [48]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2)
classifier = LogisticRegressionCV(penalty='l2', solver='saga', Cs=[6.5555])
classifier.fit(train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(test_data, test_labels))
print('proba', proba_metric(classifier, test_data, test_labels))

0.5819037872453589
0.5825892857142857
proba 0.47633565622518986


Попробуем другие признаки, присущие именно героям
==

In [49]:
columns = ['kda', 'dmg_hero_minute', 'heal_minute', 'enemy_kda', 'enemy_dmg_hero_minute', 'enemy_heal_minute']
df_vhs_params = pd.DataFrame(np.zeros((len(indexes), len(columns))), 
                      index=indexes,
                      columns=columns)

In [136]:
df_vhs_params

,kda,dmg_hero_minute,heal_minute,enemy_kda,enemy_dmg_hero_minute,enemy_heal_minute
5570720824,2.74588,573.55902,35.40042,2.6724,534.0114,2.655
5570737832,0.00000,0.00000,0.00000,0.0000,0.0000,0.000
5570740694,0.00000,0.00000,0.00000,0.0000,0.0000,0.000
5570717826,0.00000,0.00000,0.00000,0.0000,0.0000,0.000
5570711957,0.00000,0.00000,0.00000,0.0000,0.0000,0.000
...,...,...,...,...,...,...
5579886232,0.00000,0.00000,0.00000,0.0000,0.0000,0.000
5579919719,0.00000,0.00000,0.00000,0.0000,0.0000,0.000
5579821089,0.00000,0.00000,0.00000,0.0000,0.0000,0.000
5579882745,0.00000,0.00000,0.00000,0.0000,0.0000,0.000


In [50]:
columns = ['kda', 'dmg_hero_minute', 'heal_minute']
for ind in indexes:
    for param in columns:
        for hero in data_vhs.loc[ind]['radiant']:
            df_vhs_params.loc[ind][param] += heroes[hero][param] / 5
        for en_hero in data_vhs.loc[ind]['dire']:
            df_vhs_params.loc[ind]['enemy_' + param] += heroes[en_hero][param] / 5

In [51]:
df_vhs_params['winrate'] = 0.
df_vhs_params['enemy_winrate'] = 0.

for ind in indexes:
    for hero in data_vhs.loc[ind]['radiant']:
        df_vhs_params.loc[ind]['winrate'] += (heroes[hero]['winrate']['>5K MMR'] - 50) / 5
    for hero in data_vhs.loc[ind]['dire']:
        df_vhs_params.loc[ind]['enemy_winrate'] += (heroes[hero]['winrate']['>5K MMR'] - 50) / 5

In [52]:
df_vhs_params['team_wr'] = 0.
df_vhs_params['dmg_tower_minute'] = 0.
df_vhs_params['enemy_dmg_tower_minute'] = 0.

for ind in indexes:
    for hero in data_vhs.loc[ind]['radiant']:
        df_vhs_params.loc[ind]['dmg_tower_minute'] += heroes[hero]['dmg_tower_minute'] / 5
        for enemy_hero in data_vhs.loc[ind]['dire']:
            df_vhs_params.loc[ind]['team_wr'] += heroes[hero]['advantages'][enemy_hero] / 25
    for en_hero in data_vhs.loc[ind]['dire']:
        df_vhs_params.loc[ind]['enemy_dmg_tower_minute'] += heroes[en_hero]['dmg_tower_minute'] / 5

In [53]:
df_vhs_params['win'] = 0.

for ind in indexes:
    df_vhs_params.loc[ind]['win'] = df_vhs.loc[ind]['win']

In [160]:
df_vhs_params

,kda,dmg_hero_minute,heal_minute,enemy_kda,enemy_dmg_hero_minute,enemy_heal_minute,winrate,enemy_winrate,team_wr,dmg_tower_minute,enemy_dmg_tower_minute,win
5570720824,2.74588,573.55902,35.40042,2.88230,517.85076,6.82528,-0.21836,1.49980,-0.006448,88.87212,56.19738,1.0
5570737832,3.45558,749.28012,5.23154,2.54808,494.37106,17.49616,1.20670,-1.56780,0.500100,69.95428,70.52992,1.0
5570740694,2.84840,451.72070,65.48976,3.14592,562.92524,37.73410,-0.37710,1.21698,0.026108,58.34932,86.07426,0.0
5570717826,3.06166,517.27858,22.87714,2.87838,563.66988,42.17618,1.11316,-0.52436,0.002732,55.93950,68.88430,1.0
5570711957,3.16182,576.92504,8.35826,3.05212,645.59886,3.97288,-0.61638,-1.64984,0.174228,55.39800,50.22402,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5579886232,2.79728,598.58204,3.28020,2.47360,527.42054,4.73076,0.39550,-1.51630,0.094208,85.44546,73.94524,1.0
5579919719,3.12784,617.43442,15.93774,2.92894,596.58806,34.19326,1.00576,-1.76604,0.359360,51.36418,55.59968,0.0
5579821089,2.85156,511.92710,54.85050,2.70234,644.04784,19.44654,0.76868,-1.17342,-0.440896,61.41216,49.59128,1.0
5579882745,2.70084,473.58054,27.53504,2.58688,548.06106,23.34674,0.62990,-0.06572,0.156784,42.99414,102.48850,0.0


**На сей раз имеет смысл произвести масштабирование**

In [54]:
X = df_vhs_params.values[:,0:-1]
y = df_vhs_params.values[:,-1]
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, test_size = 0.2,random_state = 0)

In [55]:
std = StandardScaler()
std.fit(train_data, train_labels)
std_train_data = std.transform(train_data)
std_test_data = std.transform(test_data)

**L1**

In [163]:
%%time
gridL1(std_train_data, train_labels)

0.5753226369026858
{'C': 1.2000000000000002, 'penalty': 'l1', 'solver': 'saga'}
Wall time: 16.1 s


In [164]:
%%time
gridC('l1', std_train_data, train_labels, 'saga', np.linspace(0.5, 1.5, 10))

0.5744157656086502
{'C': 0.6111111111111112}
Wall time: 12.9 s


In [56]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2)
classifier = LogisticRegressionCV(penalty='l1', solver='saga', Cs=[0.6111])
classifier.fit(std_train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(std_test_data, test_labels))
print('proba', proba_metric(classifier, std_test_data, test_labels))

0.5768111127720476
0.5811941964285714
proba 0.4820522067119252


**L2**

In [166]:
%%time
gridL2(std_train_data, train_labels)

0.5727764213463551
{'C': 0.1, 'penalty': 'l2', 'solver': 'sag'}
Wall time: 32.4 s


In [167]:
%%time
gridC('l2', std_train_data, train_labels, 'sag', np.linspace(0.0001, 1, 10))

0.5758807115451692
{'C': 0.2223}
Wall time: 13.5 s


In [57]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2)
classifier = LogisticRegressionCV(penalty='l2', solver='sag', Cs=[0.5759])
classifier.fit(std_train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(std_test_data, test_labels))
print('proba', proba_metric(classifier, std_test_data, test_labels))

0.5767064616085493
0.5816127232142857
proba 0.4820350940610941


**В выборке не так много признаков, поэтому можем добавть признаки второй степени**

In [58]:
df_vhs_params_sq = df_vhs_params.iloc[:,:-1].copy()
for i in range(df_vhs_params.iloc[:,:-1].shape[1]):
    for j in range(i + 1):
        df_vhs_params_sq[df_vhs_params_sq.columns[i] + df_vhs_params_sq.columns[j]] = df_vhs_params_sq.iloc[:, i] * df_vhs_params_sq.iloc[:, j]

In [59]:
df_vhs_params_sq['win'] = 0.

for ind in indexes:
    df_vhs_params_sq.loc[ind]['win'] = df_vhs.loc[ind]['win']

In [60]:
X = df_vhs_params_sq.values[:,0:-1]
y = df_vhs_params_sq.values[:,-1]
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, test_size = 0.2,random_state = 0)

In [61]:
std = StandardScaler()
std.fit(train_data, train_labels)
std_train_data = std.transform(train_data)
std_test_data = std.transform(test_data)

**L1**

In [180]:
%%time
gridL1(std_train_data, train_labels)

0.5774851761423089
{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Wall time: 32min 25s


In [181]:
%%time
gridC('l1', std_train_data, train_labels, 'liblinear', np.linspace(0.00001, 1, 10))

0.5779211719567494
{'C': 0.22223}
Wall time: 12min 57s


In [62]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2)
classifier = LogisticRegressionCV(penalty='l1', solver='liblinear', Cs=[0.2222])
classifier.fit(std_train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(std_test_data, test_labels))
print('proba', proba_metric(classifier, std_test_data, test_labels))

0.575171732005655
0.5795200892857143
proba 0.48194405251563316


**L2**

In [183]:
%%time
gridL2(std_train_data, train_labels)

0.5737356121381234
{'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}
Wall time: 10min 20s


In [184]:
%%time
gridC('l2', std_train_data, train_labels, 'saga', np.linspace(0.00001, 1, 10))

0.5775200558074643
{'C': 0.11112}
Wall time: 3min 41s


In [63]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2)
classifier = LogisticRegressionCV(penalty='l2', solver='saga', Cs=[0.11112])
classifier.fit(std_train_data, train_labels)
print(classifier.scores_[1].mean())
print(classifier.score(std_test_data, test_labels))
print('proba', proba_metric(classifier, std_test_data, test_labels))

0.574962241074155
0.5802176339285714
proba 0.48190301401865276


Новая метрика
==

**Результаты всё ещё оставляют желать лучшего, но давайте попробуем немного другую метрику качества, посмотрим, насколько в среднем ошибается наша модель в предсказании.**

In [19]:
def proba_metric(classifier, test_data, test_labels):
    return sum(abs(classifier.predict_proba(test_data)[:,1] - test_labels)) / test_labels.shape[0]